<a href="https://colab.research.google.com/github/Oh-Seokjin/Text_Classifier_goorm/blob/main/BERT_with_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import requirements

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 14.2 MB/s 
     |████████████████████████████████| 163 kB 82.1 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 


In [2]:
pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 14.8 MB/s 
     |████████████████████████████████| 132 kB 82.4 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 9.2 MB 87.0 MB/s 
     |████████████████████████████████| 140 kB 94.7 MB/s 
     |████████████████████████████████| 127 kB 95.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached wandb-0.13.3-py2.py3-none-any.whl (1.8 MB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached setproctitle-1.3.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached shortuuid-1.0.9-py3-none-any.whl (9.4 kB)
  Using cached sentry_sdk-1.9.9-py2.py3-none-any.whl (162 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached sentry_sdk-1.9.8-py2.py3-none-any.whl (158 kB)
  Using cached sentry_sdk-1.9.7-py2.py3-none-any.whl (157 kB)
  Using cached sentry_sdk-1.9.6-py2.py3-none-any.whl (157 kB)
  Using cached sentry_sdk-1.9.5-py2.py3-none-any.whl (157 kB)
  Using cached sentry_sdk-1.9.4-py2.py3-none-any.whl (157 kB)
 

In [6]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict
import math

import pandas as pd
import numpy as np
from tqdm import tqdm, trange

import torch
from torch.nn.utils.rnn import pad_sequence

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

# 1. Preprocess

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Data Augmentation
# https://github.com/catSirup/KorEDA
# but 등으로 인해 문맥이 바뀔 경우는 고려하지 못하지만, 일단 시도

import random

pos_aug_wordslist = ["satisfying", "nice", "pretty good", "excellent", "friendly", "apologetic", "nice and cold", "better", "much better", "better happy", "just keep", "happy hours", "cozy", "friendly", "nostalgia", "sounds good", "include", "just ok", "zero complaints", ""]
neg_aug_wordslist = ["hard", "bad", "shit", "messed up", "who cares", "not so great", "different", "there are far superior", "geek", "complicated"]

def add_word(new_words, emotion_li):
    if len(new_words) >= 1:
        emotion_random_idx = random.randint(0, len(emotion_li)-1)
        random_word = emotion_li[emotion_random_idx]
        random_idx = random.randint(0, len(new_words)-1)
        new_words.insert(random_idx, random_word)

def random_insertion(words, n, emotion_li):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words, emotion_li)
    return new_words

In [ ]:
aug_cnt = 40000 # 증강할 문장 수 
aug_n = 3 # 문장 당 증강할 단어 수

def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join('../content/drive/MyDrive/Goorm/project01/data', file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    pos_before_aug = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/sentiment_train_1.csv')
    neg_before_aug = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/sentiment_train_0.csv')

    # train_poss
    for i in range(aug_cnt):
        words = pos_before_aug['Sentence'][i*3].split(' ')
        insert_words = random_insertion(words, aug_n, pos_aug_wordslist)
        inserted_sentence = " ".join(insert_words)
        pos_before_aug['Sentence'][i*3] = inserted_sentence

    train_pos = pos_before_aug['Sentence'].to_list()

    for i in range(len(pos_before_aug)):
        train_pos[i] = ' '.join([str(k) for k in tokenizer.encode(train_pos[i])])

    # train_neg
    for i in range(aug_cnt):
        words = neg_before_aug['Sentence'][i*3].split(' ')
        insert_words = random_insertion(words, aug_n, neg_aug_wordslist)
        inserted_sentence = " ".join(insert_words)
        neg_before_aug['Sentence'][i*3] = inserted_sentence

    train_neg = neg_before_aug['Sentence'].to_list()

    for i in range(len(neg_before_aug)):
        train_neg[i] = ' '.join([str(k) for k in tokenizer.encode(train_neg[i])])
    
    #train_pos = make_data_strings('sentiment.train.1')
    #train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [ ]:
!ls

In [ ]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

In [ ]:
train_pos[:10]
# train_neg[:10]

In [ ]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [ ]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [ ]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

In [ ]:
def collate_fn_style(samples):
    # 나누어줌, 배치로 받으니까 복수s
    input_ids, labels = zip(*samples)
    # 배치 최대 길이
    max_len = max(len(input_id) for input_id in input_ids)
    # 길이 기준으로 정렬한 index return 
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1] 
    # sorted_indices = range(len(input_ids))

    # 패딩 넣어줌, batch_first=True -> shape = B*max, batch_first=False -> shape = max*B(element-wise)
    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    
    # 길이 맞춰준 문장에서 의미 있는 토큰만 보기 위해 1, 0을 통해 살릴 부분 결정
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [ ]:
collate_fn_style(train_dataset)

In [ ]:
train_batch_size=32
eval_batch_size=32

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [ ]:
# random seed
random_seed=33
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

In [ ]:
model.train()
learning_rate = 3e-5
optimizer = AdamW(model.parameters(),lr= learning_rate)

In [ ]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [ ]:
wandb.init()

train_epoch = 3
lowest_valid_loss = 9999.
for epoch in range(train_epoch):
    with tqdm(train_loader, unit="batch") as tepoch:
        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)

            optimizer.zero_grad()

            output = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           token_type_ids=token_type_ids,
                           position_ids=position_ids,
                           labels=labels)

            loss = output.loss
            loss.backward()

            optimizer.step()

            tepoch.set_postfix(loss=loss.item())
            if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                # Evaluate the model five times per epoch
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                desc='Eval',
                                                                                                position=1,
                                                                                                leave=None):
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       position_ids=position_ids,
                                       labels=labels)

                        logits = output.logits
                        loss = output.loss
                        valid_losses.append(loss.item())

                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        predictions += batch_predictions
                        target_labels += batch_labels

                acc = compute_acc(predictions, target_labels)
                valid_loss = sum(valid_losses) / len(valid_losses)
                wandb.log({
                    "Test Acc": acc,
                    "Test Loss": valid_loss,
                    "Learning Rate": learning_rate})
                if lowest_valid_loss > valid_loss:
                    print('Acc for model which have lower valid loss: ', acc)
                    torch.save(model.state_dict(), "./pytorch_model.bin")

wandb.finish()

In [ ]:
import pandas as pd
test_df = pd.read_csv('../content/drive/MyDrive/Goorm/project01/data/test_no_label.csv')

In [ ]:
test_dataset = test_df['Id']

In [ ]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [ ]:
test = make_id_file_test(tokenizer, test_dataset)

In [ ]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [ ]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [ ]:
def collate_fn_style_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)
    # sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    sorted_indices = range(len(input_ids))

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids

In [ ]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions

In [ ]:
test_df['Category'] = predictions

In [ ]:
test_df.to_csv('submission.csv', index=False)

# test before submission

In [ ]:
# best acc = 0.988
Best_submission = pd.read_csv('/content/submission_ensemble.csv')
My_submission = pd.read_csv('/content/submission.csv')

In [ ]:
check_list = []
cnt = 0

for i in range(1000):
    if Best_submission['Category'][i] != My_submission['Category'][i]:
        check_list.append(i+2)
        cnt += 1
        print(My_submission['Id'][i], My_submission['Category'][i])

print(cnt)
print(check_list)


# Ensemble

In [35]:
Submission1 = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/submission_using albert model & fix attention mask.csv')
Submission2 = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/submission_latestbydoongle.csv')
Submission3 = pd.read_csv('/content/submission.csv')
Submission4 = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/submission_electra_lrweight_decay.csv')
Submission5 = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/submission_BERT_base.csv')

test_df = pd.read_csv('../content/drive/MyDrive/Goorm/project01/data/test_no_label.csv')

In [36]:
ensemble_predictions = []

for i in range(1000):
    ensemble_prediction = math.floor((Submission1['Category'][i] +
                                      Submission2['Category'][i] +
                                      Submission3['Category'][i] + 
                                      Submission4['Category'][i] + 
                                      Submission5['Category'][i])/5)
    ensemble_predictions.append(ensemble_prediction)

In [37]:
test_df['Category'] = ensemble_predictions

In [38]:
test_df.to_csv('submission_ensemble.csv', index=False)